# Import library

In [53]:
import requests
import json
import pandas as pd
import numpy as np

In [54]:
url_fcp_swagger = "https://fr1pslcmf05:8770/api/"
url_pricing_config = "https://fr1pslcmf05:8770/api/pricing/configs"
url_result_handler ="https://fr1pslcmf05:8770/api/pricing/report/result/Pricing"
url_measureGRoup = "https://fr1pslcmf05:8770/api/pricing/measureGroups"
url_fxoptionTrade = "https://fr1pslcmf05:8770/api/pricing/Store/Trade/FxOption"

# Mandatory field : we use those defined in FCP no need to define it

In [55]:
configName="DEFAULT"
pricingMethod="THEORETICAL"
marketDataSetId="$id/DEFAULT"
marketDataProviderId="PE_STORE_MDP"
resultHandlerConfigId=None
resultHandlerId="Collector"

# Helper function for REST call : GET / PUT / POST /DELETE

In [56]:
#helper
def get(url):
    """REST CALL : GET"""
    reponse = requests.get(url, verify=False)
    if (reponse.ok):
        val = reponse.json()
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def delete(url ):
    """REST CALL : DELETE"""
    reponse = requests.delete(url,verify=False)
    if (reponse.ok):
        val = "data deleted"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val


def put(url, json):
    """REST CALL : PUT"""
    reponse = requests.put(url, json=json, verify=False)
    if (reponse.ok):
        val = "build"
    else:
        status = reponse.raise_for_status()
        val = "error : " + status
    reponse.close()
    return val

def post(url, json):
    """REST CALL : POST"""
    reponse = requests.post(url, json=json, verify=False,headers={'Connection':'close'})
    if(reponse.ok):
        if(reponse.status_code==201):
            val = reponse.headers._store['location'][1]
        else:
            val = reponse.json()
    elif(reponse.status_code==400):
        val= reponse.json()
    else:
        status = reponse.raise_for_status()
        val =  "error : " + status
    reponse.close()
    return val



In [57]:
def getPricingId(configName):
    urlConfig = "https://fr1pslcmf05:8770/api/pricing/configs"
    config = get(urlConfig)
    for item in config:
        if (item['name']==configName):
            return item["id"]
    return "error"

In [58]:
def unitary(aod,referenceCurrency,scenarioContexts,perimeter,configName =configName, pricingMethod = pricingMethod,marketDataSetId = marketDataSetId,marketDataProviderId=marketDataProviderId,resultHandlerId = resultHandlerId,resultHandlerConfigId=resultHandlerConfigId):
    unitary_price = {}
    unitary_price["asOfDate"] = aod
    unitary_price["referenceCurrency"] = referenceCurrency     
    unitary_price["pricingMethod"] = pricingMethod
    unitary_price["marketDataSetId"] = marketDataSetId
    unitary_price["resultHandlerId"] = resultHandlerId
    unitary_price["resultHandlerConfigId"] = resultHandlerConfigId
    unitary_price["marketDataProviderId"] = marketDataProviderId    
    unitary_price["pricingConfigId"] = getPricingId(configName)
    unitary_price["perimeter"] = perimeter
    unitary_price["scenarioContexts"] = scenarioContexts
    url_price = url_fcp_swagger + "pricing/price/"
    reponse = post(url=url_price, json=unitary_price)
    return reponse["pricingResults"]


# NPV use case

In [59]:
#input needed : that need to be defined
scenarioContexts =[
    {
      "id": "base",
      "measureGroupIds": [
        "NPV"
      ]
    }
]
perimeter = {"trade" : {"ids": ["FXOptionVanilla_2"]}}

In [60]:
resultJson = unitary(aod="2016-07-04",referenceCurrency="$id/USD",perimeter=perimeter, scenarioContexts = scenarioContexts)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [61]:
resultJson

{'FXOptionVanilla_2': {'currencyId': '$id/USD',
  'pricerId': {'library': 'FQPFXDerivative',
   'pricingModel': 'Black Scholes',
   'numericalMethod': 'Closed Formula'},
  'scenarios': [{'id': 'base',
    'entries': [{'id': '0',
      'measures': {'NPV': 43597842.8377264},
      'properties': {},
      'cashFlows': [],
      'legs': []}]}]}}

   # create a measureGRoup

In [62]:
id_measure_group = "FXO_TC_ID"
measureGroupTD = {
    "id" : "FXO_TC_ID", 
    "name": "FXO_TRADE_CAPTURE",
     "measureIds": [
         "NPV",
         "FXD_Delta_Forward",
         "FXD_Delta_Spot",
         "FXD_Gamma",
         "FXD_Theta",
         "FXD_Rho_Domestic",
         "FXD_Rho_Foreign",
         "FXD_Vega",         
     ]
}

In [63]:
post(url_measureGRoup,measureGroupTD)  

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'https://fr1pslcmf05:8770/api/pricing/measureGroups/FXO_TC_ID'

In [64]:
get(url_measureGRoup+"/"+id_measure_group)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'id': 'FXO_TC_ID',
 'name': 'FXO_TRADE_CAPTURE',
 'measureIds': ['FXD_Delta_Forward',
  'FXD_Delta_Spot',
  'FXD_Gamma',
  'FXD_Rho_Domestic',
  'FXD_Rho_Foreign',
  'FXD_Theta',
  'FXD_Vega',
  'NPV']}

In [65]:
#input needed : that need to be defined
scenarioContexts =[
    {
      "id": "base",
      "measureGroupIds": [
        id_measure_group
      ]
    }
]
perimeter = {"trade" : {"ids": ["FXOptionVanilla_2"]}}

In [66]:
resultJson = unitary(aod="2016-07-04",referenceCurrency="$id/USD",perimeter=perimeter, scenarioContexts = scenarioContexts)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [23]:
resultJson

{'FXOptionVanilla_2': {'currencyId': '$id/USD',
  'pricerId': {'library': 'FQPFXDerivative',
   'pricingModel': 'Black Scholes',
   'numericalMethod': 'Closed Formula'},
  'scenarios': [{'id': 'base',
    'entries': [{'id': '0',
      'measures': {'FXD_Gamma': 2.6029829728034266e-09,
       'FXD_Theta': -22535957569659.08,
       'NPV': 43597842.8377264,
       'FXD_Vega': 8.87717787817909e-12,
       'FXD_Delta_Forward': -250022657.8402953,
       'FXD_Rho_Foreign': 12226823.015886612,
       'FXD_Delta_Spot': -250066702.57299024,
       'FXD_Rho_Domestic': 477827867172365.75},
      'properties': {},
      'cashFlows': [],
      'legs': []}]}]}}

In [24]:
delete(url_measureGRoup+"/"+"FXO_TC_ID")

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'data deleted'

In [25]:
get(url_measureGRoup)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{'id': 'NPV', 'name': 'NPV', 'readOnly': True},
 {'id': 'Standard', 'name': 'Standard', 'readOnly': True},
 {'id': 'Advanced', 'name': 'Advanced', 'readOnly': True}]

# FX OPTION TRADE ID

In [26]:
dic_id={}

In [27]:
fxoTrades = get(url_fxoptionTrade)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [28]:
def changeTrade(trade):
    old_id = trade["id"]
    currency = trade["currency"]
    trade["id"] = "newID"
    trade["currency"]
    return trade

In [29]:
modif_trade = []
for trade in fxoTrades:
    modif_trade.append(changeTrade(trade))

In [30]:
modif_trade

[{'id': 'newID',
  'type': 'FX Option',
  'subType': 'Vanilla',
  'currency': 'JPY',
  'maturityDate': '2016-09-30',
  'businessUnit0': None,
  'businessUnit1': None,
  'businessUnit2': None,
  'businessUnit3': None,
  'businessUnit4': None,
  'counterparty': None},
 {'id': 'newID',
  'type': 'FX Option',
  'subType': 'Vanilla',
  'currency': 'USD',
  'maturityDate': '2016-07-18',
  'businessUnit0': None,
  'businessUnit1': None,
  'businessUnit2': None,
  'businessUnit3': None,
  'businessUnit4': None,
  'counterparty': None},
 {'id': 'newID',
  'type': 'FX Option',
  'subType': 'Vanilla',
  'currency': 'USD',
  'maturityDate': '2017-01-24',
  'businessUnit0': None,
  'businessUnit1': None,
  'businessUnit2': None,
  'businessUnit3': None,
  'businessUnit4': None,
  'counterparty': None},
 {'id': 'newID',
  'type': 'FX Option',
  'subType': 'Vanilla',
  'currency': 'GBP',
  'maturityDate': '2016-07-28',
  'businessUnit0': None,
  'businessUnit1': None,
  'businessUnit2': None,
  'busi

# FX VOL workflow

In [67]:
url_fx_vol = "https://fr1pslcmf05:8770/api/pricing/Store/MarketData/FxVolatilityCurves"

In [68]:
dropdown = get(url_fx_vol)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [69]:
dropdown

[{'id': '$id/CAD_JPY_VolSurf', 'name': 'CAD_JPY_VolSurf'},
 {'id': '$id/EUR_GBP_VolSurf', 'name': 'EUR_GBP_VolSurf'},
 {'id': '$id/EUR_HKD_VolSurf', 'name': 'EUR_HKD_VolSurf'},
 {'id': '$id/EUR_JPY_VolSurf', 'name': 'EUR_JPY_VolSurf'},
 {'id': '$id/EUR_USD_VolSurf', 'name': 'EUR_USD_VolSurf'},
 {'id': '$id/GBP_USD_VolSurf', 'name': 'GBP_USD_VolSurf'},
 {'id': '$id/USD_JPY_VolSurf', 'name': 'USD_JPY_VolSurf'}]

In [70]:
name = "EUR_USD_VolSurf"
url_fx_value = "https://fr1pslcmf05:8770/api/pricing/Store/MarketData/FxVolatilityCurveValues?name="+name
url_fx_def = "https://fr1pslcmf05:8770/api/pricing/Store/MarketData/FxVolatilityCurveDefinitions?name="+name

In [71]:
id_fx_value = get(url_fx_value)
print(id_fx_value) #choix par rapport à la date 

[{'id': '59efd157d2cb4725a996f46aa052a6e3', 'marketSet': 'DEFAULT', 'date': '2016-07-04', 'name': 'EUR_USD_VolSurf'}]


C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [72]:
id_fx_def = get(url_fx_def)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [73]:
url_val = "https://fr1pslcmf05:8770/api/pricing/Store/MarketData/FxVolatilityCurveValues"+"/"+str(id_fx_value[0]['id'])
url_def = "https://fr1pslcmf05:8770/api/pricing/Store/MarketData/FxVolatilityCurveDefinitions"+"/"+str(id_fx_def[0]['id'])

In [74]:
defFx = get(url_def)

C:\Users\jerom\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [75]:
defFx

{'id': '777c2ce0d55547c08319401d3e6157ac',
 'name': 'EUR_USD_VolSurf',
 'marketSet': 'DEFAULT',
 'currencyPair': 'EUR/USD',
 'domesticCurve': 'USD-FFUND',
 'foreignCurve': 'EUR_XUSD',
 'volatilityBasis': 'ACT/365.FIXED',
 'premiumAdjusted': False,
 'termStructureInterpolation': {'interpolationMethod': 'linear',
  'extrapolationMethodBefore': 'flat',
  'extrapolationMethodAfter': 'flat',
  'variable': 'volatility'},
 'smileInterpolation': {'interpolationMethod': 'linear',
  'extrapolationMethod': 'flat',
  'variable': 'volatility'}}

In [76]:
id_fx_value

[{'id': '59efd157d2cb4725a996f46aa052a6e3',
  'marketSet': 'DEFAULT',
  'date': '2016-07-04',
  'name': 'EUR_USD_VolSurf'}]